### IMPORT LIBRARY

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate_update import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from termcolor import colored
from send_email import *

### CONNECT TO AZURE SQL

In [ ]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test ConnectionD
try:
    result = engine.execute("SELECT 1")
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

### CONNECT TO SHAREPOINT

In [ ]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

### READ AND LIST FOLDER

In [ ]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

In [ ]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

In [ ]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

In [ ]:
#Provide information
Province_name=['Binh Duong Province']#Chọn tỉnh cần import-- là tên folder 
Year=['2023']#Chọn năm cần import
Quarter=['Q1']#Chọn quý cần import
#Có thể chọn nhiều tỉnh, tên tỉnh để trong dấu nháy đơn và cách nhau bởi dấy phẩy. Áp dụng tương tự cho năm, quý
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    if i.split('/')[-3] in Province_name and i.split('/')[-2] in Year and i.split('/')[-1] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

### GET IMPORT FILE

In [ ]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[1].upper()
    if sector in ('VLTH'): #Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
        url.append(i)
    else:
        pass
url

### IMPORT DATA

In [ ]:
'''Prepare ingredients''' 
columns_that_need_unidecode = ['Sub_Project_Name', 'Project_City_Name', 'Project_District_Name']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
for i in name_sector:
    globals()[f'df_update_{i.lower()}'] = pd.DataFrame([])
    globals()[f'df_flat__{i.lower()}'] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    # #-------------------------------------------------------

    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)

    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Type', 'Grade']
    lst_cls = ['Project_City_Name', 'Project_District_Name', 'Sub_Project_Type', 'Grade']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        if flag_key == 0:
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            if sector == 'RETAIL':
                df_update_retail = pd.concat([df_update_retail, data], axis=0)
                df_flat_retail = get_flat_file(df_update_retail, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_retail, df_update_retail)
            elif sector == 'OFFICE':
                df_update_office = pd.concat([df_update_office, data], axis=0)
                df_flat_office = get_flat_file(df_update_office, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_office, df_update_office)
            elif sector == 'HOTEL':
                df_update_hotel = pd.concat([df_update_hotel, data], axis=0)
                df_flat_hotel = get_flat_file(df_update_hotel, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_hotel, df_update_hotel)
            elif sector == 'SA' or sector=='SERVICED_APARTMENT':
                df_update_sa = pd.concat([df_update_sa, data], axis=0)
                df_flat_sa = get_flat_file(df_update_sa, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_sa, df_update_sa)
            elif sector == 'APT' or sector=='APARTMENT':
                df_update_apt = pd.concat([df_update_apt, data], axis=0)
                df_flat_apt = get_flat_file(df_update_apt, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_apt, df_update_apt)
            elif sector == 'VLTH':
                df_update_vlth = pd.concat([df_update_vlth, data], axis=0)
                df_flat_vlth = get_flat_file(df_update_vlth, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_vlth, df_update_vlth)
            #-------------------------------------------------------    
            if flag_update == 0:
#                 insert_to_fresh(file_url, data, cnt_str)
                list_df_update = [df_update_retail, df_update_office, df_update_hotel, df_update_sa, df_update_apt, df_update_vlth]
                for df_update in list_df_update:
                    if len(df_update) != 0:
                        df_update = Generate_Additional_Columns(df_update,url,df_summ_file,BIHub,engine,file_url)
                        insert_to_tracking(df_update, sector, engine)
            else:
                print(colored(f'Some keys wrong in {file_name}', 'yellow'))
        else:
            print(colored(f'Cant get key in {file_name}', 'yellow'))
    else:
        pass

In [ ]:
#Test check_project_key
'''Prepare ingredients''' 
columns_that_need_unidecode = ['Sub_Project_Name', 'Project_City_Name', 'Project_District_Name']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
for i in name_sector:
    globals()[f'df_update_{i.lower()}'] = pd.DataFrame([])
    globals()[f'df_flat__{i.lower()}'] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    # #-------------------------------------------------------

    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)

    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Type', 'Grade']
    lst_cls = ['Project_City_Name', 'Project_District_Name', 'Sub_Project_Type', 'Grade']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''

list_keys = ['Sector', 'Sub_Project_Name', 'Project_District_Name','Project_City_Name','Project_Type','Project_Key']
list_keys_on =['Sector', 'Sub_Project_Name','Project_District_Name','Project_City_Name','Project_Type']
table_name = 'Main_Sector_Project'
lower_function = lambda x: x.strip().lower() if isinstance(x, str) else x
if sector in ['VLTH', 'RETAIL']:
    data = data.rename(columns = {'Sub_Project_Type':'Project_Type'})

if sector == 'VLTH':
    if 'Branded_Flag' in data.columns:
        df_query = pd.read_sql(f"select * from FRESH.{table_name} WHERE Branded_Flag = 1",engine)
    else:
        df_query = pd.read_sql(f"select * from FRESH.{table_name}",engine)
data[list_keys_on] = data[list_keys_on].applymap(lower_function)
df_query[list_keys] = df_query[list_keys].applymap(lower_function)
merged_data = pd.merge(data, df_query[list_keys],
                        left_on = list_keys_on,
                        right_on = list_keys_on,
                        how = 'left'
                    )
# merged_data                    
processed_data = merged_data[merged_data['Project_Key'].isnull()]
processed_data.count().sum()

### End check_project_key

# Test update_flat_file

In [ ]:
df_flat = df_flat_vlth
df_flat.head()
# df_update = df_update_vlth
# flag_update = 0

In [ ]:
df_update = df_update_vlth
df_update

In [ ]:
#list_update
flag_update = 0
list_keys_on = ['Project_Key', 'Date_Key']
if sector == 'RETAIL':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'GFA', 'NLA', 'Vacant_Area', 'Leased_Area'
                    , 'Avg_Gross_Rent', 'New_Supply'
                    ]
elif sector == 'OFFICE':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'NLA', 'Leased_Area_End_Of_Q', 'Vacant_Area'
                    , 'Avg_Net_Rent', 'Avg_Gross_Rent', 'New_Supply'
                    ]
elif sector == 'HOTEL':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'Total_Supply', 'Leased_Rooms', 'Avg_Room_Price'
                    , 'Quoted_Room_Rate', 'Occupancy', 'New_Supply'
                    ]
elif sector == 'SERVICED_APARTMENT' or sector == 'SA':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'Total_Supply', 'Launch_Units', 'Leased_Units', 'Vacant_Units'
                    , 'Avg_Rent', 'Net_Lettable', 'Rev_PAU', 'Rev_PAU_m2', 'Rent_Per_Unit', 'New_Supply'
                    ]
elif sector == 'APT' or sector=='APARTMENT':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'Total_Supply', 'Secondary_Supply', 'Newly_Launch', 'Total_Launched_Units'
                    , 'Available_Units', 'Total_Sold_Units','Sold_Units_End_Of_Q', 'Quarterly_Sales'
                    , 'Remaining_Supply', 'NFA_Primary_Price', 'NSA_Primary_Price', 'UNIT_Primary_Price'
                    , 'Avg_Primary_Price', 'NFA_Secondary_Price', 'NSA_Secondary_Price', 'UNIT_Secondary_Price'
                    , 'Avg_Secondary_Price', 'Quarterly_Sales_New_Supply'
                    ]
elif sector == 'VLTH':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Project_Status'
                    , 'Total_Supply', 'Newly_Launch', 'Total_Launched_Units', 'Available_Units'
                    , 'Total_Sold_Units', 'Sold_Units_End_Of_Q', 'Quarterly_Sales', 'Remaining_Supply'
                    , 'LA_Primary_Price', 'GFA_Primary_Price', 'UNIT_Primary_Price', 'LA_Secondary_Price'
                    , 'GFA_Secondary_Price', 'UNIT_Secondary_Price', 'Quarterly_Sales_New_Supply'
                    ]
elif sector == 'IP':
    list_update = ['Project_Key', 'Date_Key', 'Form', 'Status'
                    , 'PROJECT_Land_Area', 'LAND_Leaseable_Land_Area', 'LAND_Available_Land_Area', 'LAND_Leased_Land_Area'
                    , 'RB_Land_Area', 'RB_Total_GFA', 'RB_NLA', 'RB_Leased_Area', 'Occupancy'
                    , 'LAND_AVG_Rent_Primary_Price', 'LAND_AVG_Rent_Secondary_Price', 'LAND_AVG_Rent_Actual'
                    , 'LAND_AVG_Rent_Last_Transaction', 'RB_AVG_Rent_Primary_Price', 'RB_AVG_Rent_Secondary_Price'
                    , 'RB_AVG_Rent_Actual', 'RB_AVG_Rent_Last_Transaction'
                    ]
list_update

In [ ]:
int_function = lambda x: int(x) if isinstance(x, str) else x
df_update[list_keys_on] = df_update[list_keys_on].applymap(int_function)
df_flat[list_keys_on] = df_flat[list_keys_on].applymap(int_function)
merge_how = ['left', 'inner']    
for i in merge_how:
    globals()[f'df_merge_{i.lower()}'] = pd.merge(df_update, df_flat[list_update]
                            , left_on = list_keys_on
                            , right_on = list_keys_on
                            , how = i
                        )

In [ ]:
df_merge_left[list_keys_on]

In [ ]:
df_merge_inner[list_keys_on]

In [ ]:
df_merge_gap = df_merge_left[~df_merge_left['Project_Key'].isin(df_merge_inner['Project_Key'])]
column_gap = ['Project_Key','Date_Key','Sub_Project_Name']
df_merge_gap[column_gap]